# SocialAL Model
### KLS 5.30.19 updated 8.23.19

#### Project info: https://osf.io/b48n2/
#### Model modified from :

Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version:

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## Setup

### Load modules

In [2]:
import numpy as np
import random
import math

### Initialize free parameters
**a_gain**: gain learning rate (varies 0-1)  
**a_loss**: loss learning rate (varies 0-1)  
**beta**: inverse temperature paramter (varies 0-1)

In [3]:
a_gain=0.2
a_loss=0.2
beta=1

### Initialize other variables
Three potential trustees: A, B, C [Trustworthy, Neutral, Untrustworthy?]  
**ProbA, ProbB, ProbC** = Belief about likelihood of reciprocation  
**actionProb** = Probability of a participant investing with the trustee  
**EvA, EvB, EvC** = Expected value of each potential action towards a partner  
**choice** = The choice (how much to entrust) taken by the participant  
**recip** = Whether or not the trustee reciprocated (y in paper)

In [4]:
ProbA = 0.5
ProbB = 0.5
ProbC = 0.5
actionProb = 0.5
EvA = [9,9,9,9]
EvB = [9,9,9,9]
EvC = [9,9,9,9]
choice = 2
recip = 0

### Declare constants
Four potential actions: 1, 2, 3, 4  
1 = $\$$0 money is entrusted with trustee  ( $\$$9 retained, $\$$0 potentially reciprocated)  
2 = $\$$3 is entrusted with trustee ( $\$$6 retained, $\$$6 potentially reciprocated)  
3 = $\$$6 is entrusted with trustee ( $\$$3 retained, $\$$12 potentially reciprocated)  
4 = $\$$9 is entrusted with trustee ( $\$$0 retained, $\$$18 potentially reciprocated)

In [5]:
invest = [0,3,6,9]
retain = [9-x for x in invest]
print ("Retain list is: ", retain)
shared = [2*x for x in invest]
print ("Shared list is: ", shared)

Retain list is:  [9, 6, 3, 0]
Shared list is:  [0, 6, 12, 18]


## Define update functions

### Update Values

In [20]:
def update_value(Prob, EV, choice):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
  
    EV[choice-1] = retain[choice-1] + Prob*shared[choice-1]
    return EV

### Update probabilities

In [40]:
def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

## Test update functions

### Test updating on Trustee A (reciprocates 93% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,0]
random.shuffle(recip)

for i in range(1,len(choice)):
    ProbA = update_prob(recip[i],ProbA, a_gain, a_loss)
    EvA = update_value(ProbA, EvA, choice[i])
    
print("Ending probability for Trustee A is: ", ProbA)
print("Ending expected values for Trustee A: ", EvA)

### Test updating on Trustee B (reciprocates 60% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0]
random.shuffle(recip)

for i in range(1,len(choice)):
    ProbB = update_prob(recip[i],ProbB, a_gain, a_loss)
    EvB = update_value(ProbB, EvB, choice[i])
print("Ending probability for Trustee B is: ", ProbB)
print("Ending expected values for Trustee B: ", EvB)

### Test updating on Trustee C (reciprocates 7% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
random.shuffle(recip)

for i in range(1,len(choice)):
    ProbC = update_prob(recip[i],ProbC, a_gain, a_loss)
    EvC = update_value(ProbC, EvC, choice[i])
    
print("Ending probability for Trustee B is: ", ProbC)
print("Ending expected values for Trustee B: ", EvC)

## Define action selection functions

### Softmax action selection
_update with code from Jess_

In [ ]:
def get_action_selection_prob(beta, EV, choice):
    numerator = np.exp(beta*EV[choice-1])
    #print "Numerator is: ", numerator
    denominator = np.sum([np.exp(beta*x) for x in EV])
    #print "Denominator is:", denominator
    actionProb = numerator/denominator
    #print ("Action is:", choice)
    #print ("Probability of investing: ", actionProb)
    return actionProb

In [ ]:
def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

In [ ]:
def action_selection(actionProb):
    cumprob = np.cumsum(actionProb) # cumulative probability 
    num = random.uniform(0,1)# pick a random number between 0 and 1, see where it falls
    if num < cumprob[0]: action = 1
    elif num < cumprob[1]: action = 2
    elif num < cumprob[2]: action = 3
    else: action = 4   
    return action

### Test action selection functions

In [ ]:
#print(beta)
ev = EvB # Change between EvA, EvB, and EvC to see the probability for different partners
print("Choice 1 probability is: ", get_action_selection_prob(beta, ev, 1))
print("Choice 2 probability is: ", get_action_selection_prob(beta, ev, 2))
print("Choice 3 probability is: ", get_action_selection_prob(beta, ev, 3))
print("Choice 4 probability is: ", get_action_selection_prob(beta, ev, 4))


In [ ]:
probsA = get_action_selection_probs(beta, EvA)
print(probsA)
probsB = get_action_selection_probs(beta, EvB)
print(probsB)
probsC = get_action_selection_probs(beta, EvC)
print(probsC)

In [ ]:
actionA = action_selection(probsA)
print("Action for Person A = ", actionA)
actionB = action_selection(probsB)
print("Action for Person B = ", actionB)
actionC = action_selection(probsC)
print("Action for Person C = ", actionC)

### Calculate likelihood

**totalLLH** = total log likelihood for function

In [ ]:
totalLLH = 0

In [ ]:
def get_likelihood_action(choice, EV, beta, Probs, totalLLH):
    prob = Probs[choice-1]
    totalLLH += -math.log(prob)
    return totalLLH

In [ ]:
def get_likelihood(choices, EV, beta, Probs, totalLLH):
    trials = range(0,len(choices))
    for t in trials:
        totalLLH = get_likelihood_action(choices[t], EV, beta, Probs, totalLLH)
    return totalLLH

### Test likelihood functions

In [ ]:
totalLLH = get_likelihood_action(1, EvA, beta, probsA, totalLLH)
print(totalLLH)

In [ ]:
totalLLH = 0
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
LL = get_likelihood(choice, EvA, beta, probsA, totalLLH)
print(LL)

### Calculate LL 
need to rewrite Jess's code for alternative

In [ ]:
def logsumexp(b,dim):
    # s = logsumexp(b) by Tom Minka, from Jess Cooper
    # Returns s(i) = log(sum(exp(b(:,i)))) while avoiding numerical underflow.
    # s = logsumexp(b, dim) sums over dimension 'dim' instead of summing over rows
  

In [ ]:
def update_likelihood(lik, val, action, beta):
    lik = lik + val[action] - logsumexp(val*beta,2)
    